In [75]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

'/Users/bernat/opt/anaconda3/envs/ironhack/lib/python3.9/site-packages/chromedriver_autoinstaller/107/chromedriver'

In [76]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup
import re as re
import time
import pandas as pd
import os
import numpy as np
import requests
import pymysql
import sqlalchemy as alch
from getpass import getpass

import googletrans
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy



In [104]:
import os

url = "https://www.museodelprado.es/en/the-collection/art-works"

options = webdriver.ChromeOptions()
#options.add_argument("--incognito")
#options.add_argument("--headless")

driver = webdriver.Chrome(options = options)
driver.get(url)

WebDriverWait(driver, 15).until(EC.element_to_be_clickable((By.ID, "onetrust-accept-btn-handler"))).click()

start = time.time()
driver.execute_script("document.querySelector('footer').style.display = 'None'")
last_height = int(driver.execute_script("return document.body.scrollHeight"))

elemento = "document.querySelector('mostrable miniaturas')"
#buscador-coleccion-resultados

print(last_height)
while True:

    driver.execute_script("document.querySelector('footer').style.display = 'None';")
    #driver.execute_script("document.querySelector('buscador-coleccion-resultados').scrollTo(0, document.querySelector('buscador-coleccion-resultados').scrollHeight);")
    #time.sleep(5)
    driver.execute_script("document.querySelector('.buscador-coleccion-resultados').scrollTo(0, document.body.scrollHeight);")
    new_height = driver.execute_script("return document.querySelector('.buscador-coleccion-resultados').scrollHeight")
    #buscador-coleccion-resultados
    print(new_height)
    if new_height == last_height:
        break
    last_height = new_height
    end = time.time()
    if round(end-start) > 70:
        break
html = driver.execute_script("return document.body.outerHTML;")


#OK -> this function will give me the links of the images
images=get_image_list(html)
len(images)

os.system(f"say -v monica {len(images)}")


3033


JavascriptException: Message: javascript error: Cannot read properties of null (reading 'scrollTo')
  (Session info: chrome=107.0.5304.110)
Stacktrace:
0   chromedriver                        0x0000000102a60938 chromedriver + 4745528
1   chromedriver                        0x00000001029e2443 chromedriver + 4228163
2   chromedriver                        0x0000000102645b68 chromedriver + 441192
3   chromedriver                        0x0000000102649127 chromedriver + 454951
4   chromedriver                        0x0000000102648fa0 chromedriver + 454560
5   chromedriver                        0x0000000102649cde chromedriver + 457950
6   chromedriver                        0x00000001026bcdaa chromedriver + 929194
7   chromedriver                        0x00000001026a3d52 chromedriver + 826706
8   chromedriver                        0x00000001026bc144 chromedriver + 926020
9   chromedriver                        0x00000001026a3b83 chromedriver + 826243
10  chromedriver                    

In [89]:
print(html)

<body class="" style=""><input type="hidden" id="inpt_url_external_service" value="https://servicios.museodelprado.es/servicioexterno">
<input type="hidden" id="inpt_url_cronoscopio" value="https://www.museodelprado.es/en/the-collection/timeline"> 
      
   
   
      <meta name="apple-mobile-web-app-capable" content="yes">  
<meta name="viewport" content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no">

<link rel="apple-touch-icon" sizes="57x57" href="https://content3.cdnprado.net/imagenes/favicon/apple-touch-icon-57x57.png?v=RyyW7BB3pe">
<link rel="apple-touch-icon" sizes="60x60" href="https://content3.cdnprado.net/imagenes/favicon/apple-touch-icon-60x60.png?v=RyyW7BB3pe">
<link rel="apple-touch-icon" sizes="72x72" href="https://content3.cdnprado.net/imagenes/favicon/apple-touch-icon-72x72.png?v=RyyW7BB3pe">
<link rel="apple-touch-icon" sizes="76x76" href="https://content3.cdnprado.net/imagenes/favicon/apple-touch-icon-76x76.png?v=RyyW

In [67]:
#Functions!!!!!!!

#title
def get_title(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    word="dt"

    tags = soup.find_all(word)
    titles=[c.getText().replace(">", "").strip() for c in tags]
    #the first 44 items are from the menu -> we don't want this, then, titles are duplicated!!!!
    titles = titles[44:]
    newtitles=[]
    i=0
    while i < len(titles):
        newtitles.append(titles[i])
        i+=2
    return newtitles

#subtitle
def get_subtitle(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    word="dd"

    tags = soup.find_all(word, attrs={"class":"soporte"})
    subtitles=[c.getText().replace(">", "").strip() for c in tags]

    newsub=[]
    i=0
    while i < len(subtitles):
        newsub.append(subtitles[i])
        i+=2

    return newsub


#author
def get_author(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    word="dd"

    tags = soup.find_all(word, attrs={"class":"autor"})
    authors=[c.getText().replace(">", "").strip() for c in tags]
    newaut=[]
    i=0
    while i < len(authors):
        newaut.append(authors[i])
        i+=2
    return newaut

#images
def get_image_list(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    pics = soup.find_all("img")
    image_list=[]
    for i in range(1,len(pics)-7):
        image_list.append(pics[i].get("src"))    
    return image_list

In [66]:
#OK -> this function will give me the links of the images
images=get_image_list(html)
len(images)


18

In [5]:
#subtitle
subtitles=get_subtitle(html)


In [6]:
#author

artists = get_author(html)

In [7]:
#title

titles=get_title(html)

In [43]:
prado_df = pd.DataFrame({"title":titles, "subtitle":subtitles, "artist":artists, "img_link": images})

In [44]:
prado_df.head()

title  \
0  Christ Pantocrator held by four Angels, Hermit...   
1  Lower Part of God the Father, Hermitage of Ver...   
2  Mary Magdalene, Hermitage of Vera Cruz, Maderu...   
3  The Adoration of a Wise Man, Hermitage of Vera...   
4  Two Apostles and the Remains of a Third, Hermi...   

                                            subtitle     artist  \
0  Fresco painting on mural transferred to canvas...  Anonymous   
1  Fresco painting on mural transferred to canvas...  Anonymous   
2  Fresco painting on mural transferred to canvas...  Anonymous   
3  Fresco painting on mural transferred to canvas...  Anonymous   
4  Fresco painting on mural transferred to canvas...  Anonymous   

                                            img_link  
0  https://content3.cdnprado.net/imagenes/Documen...  
1  https://content3.cdnprado.net/imagenes/Documen...  
2  https://content3.cdnprado.net/imagenes/Documen...  
3  https://content3.cdnprado.net/imagenes/Documen...  
4  https://content3.cdnprado.net/imagenes/Documen...

In [45]:
# I will split the years, that are contained in 

subt=[]
year=[]
for line in prado_df["subtitle"]:
    newline= line.split(".  ")
    subt.append(newline[0])
    year.append(newline[-1])

prado_df["subtitle"] = subt
prado_df["year"]= year

prado_df["year"] = [line.replace (" century","")for line in prado_df["year"]]


In [47]:
prado_df.head()

title  \
0  Christ Pantocrator held by four Angels, Hermit...   
1  Lower Part of God the Father, Hermitage of Ver...   
2  Mary Magdalene, Hermitage of Vera Cruz, Maderu...   
3  The Adoration of a Wise Man, Hermitage of Vera...   
4  Two Apostles and the Remains of a Third, Hermi...   

                                         subtitle     artist  \
0  Fresco painting on mural transferred to canvas  Anonymous   
1  Fresco painting on mural transferred to canvas  Anonymous   
2  Fresco painting on mural transferred to canvas  Anonymous   
3  Fresco painting on mural transferred to canvas  Anonymous   
4  Fresco painting on mural transferred to canvas  Anonymous   

                                            img_link year  
0  https://content3.cdnprado.net/imagenes/Documen...  XII  
1  https://content3.cdnprado.net/imagenes/Documen...  XII  
2  https://content3.cdnprado.net/imagenes/Documen...  XII  
3  https://content3.cdnprado.net/imagenes/Documen...  XII  
4  https://content3.cdnprado.net/imagenes/Documen...  XII

### I will transform centuries to years!

In [48]:
centuries = {"VIII":700,"IX":800,"X":900,"XI":1000,"XII":1100,"XIII":1200,"XIV":1300,"XV":1400,"XVI":1500,"XVII":1600,"XVIII":1700,"XIX":1800,
"XX":1900,"XXI":2000}

In [51]:
nl=[]
for y in prado_df["year"]:
    for key in centuries:
        if y==key:
            y= centuries[key]
            nl.append(y)
prado_df["year"]=nl

In [54]:
prado_df.head()

title  \
0  Christ Pantocrator held by four Angels, Hermit...   
1  Lower Part of God the Father, Hermitage of Ver...   
2  Mary Magdalene, Hermitage of Vera Cruz, Maderu...   
3  The Adoration of a Wise Man, Hermitage of Vera...   
4  Two Apostles and the Remains of a Third, Hermi...   

                                         subtitle     artist  \
0  Fresco painting on mural transferred to canvas  Anonymous   
1  Fresco painting on mural transferred to canvas  Anonymous   
2  Fresco painting on mural transferred to canvas  Anonymous   
3  Fresco painting on mural transferred to canvas  Anonymous   
4  Fresco painting on mural transferred to canvas  Anonymous   

                                            img_link  year  
0  https://content3.cdnprado.net/imagenes/Documen...  1100  
1  https://content3.cdnprado.net/imagenes/Documen...  1100  
2  https://content3.cdnprado.net/imagenes/Documen...  1100  
3  https://content3.cdnprado.net/imagenes/Documen...  1100  
4  https://content3.cdnprado.net/imagenes/Documen...  1100